In [5]:
import pandas as pd

data = pd.read_csv("african_crises.csv", sep = ",")
case_num = list(set(data['cc3'].tolist()))

columnns = data.columns.values.tolist()
print(columnns)
def get_subdf(case_num):
    d1 = data.loc[data['cc3'].isin([case_num])]
    #看是导致灾难的规律次序是怎么样的，看增量有没有伴随危机
    d2 = d1[["exch_usd", "inflation_annual_cpi", "gdp_weighted_default"]]
    d3 = d2.pct_change(axis='rows', periods=1)

    d3 = d3.iloc[1: , :]
    d3['unit_exch'] = d3.apply(lambda x: x["exch_usd"] / 0.0016, axis=1)
    d3['unit_cpi'] = d3.apply(lambda x: round(x["inflation_annual_cpi"], 2) / 0.2, axis=1)
    #5% gap for inflation_annual_cpi
    d4 = pd.merge(d1, d3, left_index=True, right_index=True)
    d4 = pd.merge(d4, d2, left_index=True, right_index=True)
    d4.head()
    return d4

d4 = get_subdf('EGY')
for c in case_num:
    d5 = get_subdf(c)
    d4 = d4.append(d5)

d6 = d4[["year", "cc3", "currency_crises", "inflation_crises", "banking_crisis", "sovereign_external_debt_default",
         "systemic_crisis","unit_exch", "unit_cpi", "gdp_weighted_default"]]
print(case_num)
d6.tail()
#d2 = d4[["gdp_weighted_default_x"]]


['case', 'cc3', 'country', 'year', 'systemic_crisis', 'exch_usd', 'domestic_debt_in_default', 'sovereign_external_debt_default', 'gdp_weighted_default', 'inflation_annual_cpi', 'independence', 'currency_crises', 'inflation_crises', 'banking_crisis']
['MUS', 'TUN', 'NGA', 'ZWE', 'ZAF', 'AGO', 'DZA', 'EGY', 'CAF', 'MAR', 'KEN', 'CIV', 'ZMB']


,year,cc3,currency_crises,inflation_crises,banking_crisis,sovereign_external_debt_default,systemic_crisis,unit_exch,unit_cpi,gdp_weighted_default
964,2010,ZMB,0,0,no_crisis,0,0,21.445152,-1.85,0.0
965,2011,ZMB,0,0,no_crisis,0,0,42.286895,0.10,0.0
966,2012,ZMB,0,0,no_crisis,0,0,3.988246,-1.20,0.0
967,2013,ZMB,1,0,no_crisis,0,0,45.343738,0.30,0.0
968,2014,ZMB,0,0,no_crisis,0,0,98.007246,0.60,0.0


In [6]:
def forming_states(indices, m):
    l = len(m[indices[0]])
    state = [0 for i in range(l)]
    count = 0
    for i in indices:
        count += m[i][i]
    clean = [i for i in range(l)]
    for k in indices:
        clean.remove(k)
    for i in range(l):
        if i in indices:
            state[i] = count
        else:
            for j in indices:
                state[i] += m[j][i]
    count *= -1
    prob_sum = 0
    if count == 0:
        return [0 for i in range(l)]
    for i in clean:
        
        state[i] = state[i] / count
        prob_sum += state[i]
        
    for j in clean:
        state[j] /= prob_sum
#    print("sdfsd " + str(state))
    return state


def counting_states(indices, m):
    l = len(m[indices[0]])
    state = [0 for i in range(l)]
    count = 0
    for i in indices:
        count += m[i][i]
    #print("sdf" + str(count))
    for i in range(l):
        if i in indices:
            state[i] = count
        else:
            for j in indices:
                state[i] += m[j][i]
    count *= -1
    return state
        
def get_probability_of_state(all_state_obs_indices, positives):
    state_prob = 0
    total = positives[-1]
    for obs_ind in all_state_obs_indices:
        x = positives[obs_ind]
        
        state_prob += x / total
    return state_prob
    
def get_states_hmm(all_states, positives):
    prob = []
    for states in all_states:
        state_probs = get_probability_of_state(states, positives)
        prob.append(state_probs)
    
    return prob
    
def overall_probability(states, num_nodes):
    interested = 0
    overall_event_count = 0
    for i in range(num_nodes):
        if i in states:
            interested += m[i][i]
        overall_event_count += m[i][i]
    return interested / overall_event_count

def subset_probability(states1, states2):
    num = 0
    den = 0
   # print(states1, states2, "sdf")
    for i in range(num_nodes):
        
        if i in states1:
            num += m[i][i]
        if i in states2:
            den += m[i][i]
        if den == 0:
            return 0
        #print(i, num,den)
    return max(num / den, -num / den)

from sklearn.preprocessing import normalize

def get_state_transition2(start, cprobs, all_states):
    ystates, yy = [], []
    probs = cprobs[start]
    all_prob = 0
  #  print(start)
    for s in all_states:
        cprob_s = 0
        for obs_ind in s:
         #   print(obs_ind)
            all_prob +=  probs[obs_ind]
            cprob_s += probs[obs_ind]
        ystates.append(cprob_s)

    return yy

crisis_obs = [ 37, 38,39, 12,13,14, 30, 31, 26, 27, 28, 29, 40, 4] 
crisis_obs2 = [ 40, 41, 37, 38, 39, 0, 1, 15, 16,  30, 31, 28, 29] 
crisis_obs3 = [ 40, 41, 37, 38, 39, 0, 1, 2, 3, 4, 5,  30, 31, 0, 21, 22, 23, 0] 
crisis_obs4 = [ 40, 41, 37, 38, 39, 0, 40, 41, 30, 31,0,1, 2, 3, 5]
crisis_obs5 = [ 39 , 40, 41, 10, 11, 12]
crisis_obs6 = [ 39 , 40 , 41  ,37 , 6, 7, 8, 9, 10]
downturn_response_obs = [ 12,13,14, 26, 27, 28, 29, 30 , 31]
upturn_response_obs = [ 27, 28, 29, 0,1,2, 3, 4,  32, 33]
upturn_no_crisis_obs = [ 22, 23, 24, 25, 4,5,6, 40 , 41 , 37 , 38]
downturn_no_crisis_obs = [ 22, 23, 24, 7,8,9, 40 , 41 , 37 , 38]
independence_banking_obs = [ 39 , 35,36, 35, 36, 22, 23,  1, 0, 2,3,4,5,30] 
downturn_obs = [ 22, 23, 24, 25, 7,8,9,10,11,  30, 31,  40, 41, 37, 38, 40 , 41] 
upturn_obs = [ 22, 23, 24, 25, 27, 28, 29 ,4,5,6]
updown_obs = [ 15, 16, 17, 18, 19, 20, 21, 0, 1, 2, 3, 4, 5, 6, 7] 
downup_obs = [ 22, 23, 24, 25, 26,27, 28,  8, 9, 10, 11, 12, 13, 14] 
currency_banking_crisis_obs = [30, 36, 39, 6,7,8,9,10,11] 
all_state_obs = [crisis_obs,crisis_obs2 ,crisis_obs3 ,crisis_obs4 ,crisis_obs5,crisis_obs6 ,downturn_response_obs,upturn_response_obs,upturn_no_crisis_obs,downturn_no_crisis_obs,independence_banking_obs,downturn_obs,upturn_obs,updown_obs,downup_obs,currency_banking_crisis_obs] 


#print(state_probs)
def get_state_indices(all_states, num):
    
    for i in range(len(all_states)):
        s = all_states[i]
        if num in s:
            return i

    return -1

def get_hmm2(all_states, cprobs):
    hmm = []
   # print(len(cprobs[0]))
    for ii in range(len(all_states)):
        yss = []
        for obs_indices in all_states[ii]:
            state_indices = get_state_indices(all_states, obs_indices)
            ys = get_state_transition2(state_indices, cprobs, all_states)
            yss.append(ys)
        hmm.append(yss)
    return hmm

#hmm = get_hmm2(all_states, prob_under_states)
#print(len(hmm[0]))

In [7]:
import numpy as np

def get_bucket_cpi(val):
    if val < -20:
        return 0
    elif -20 <= val < -15:
        return 1
    elif -15 <= val < -10:
        return 2
    elif -10 <= val < -8:
        return 3
    elif -8 <= val <-6:
        return 4
    elif -6 <= val < -4:
        return 5
    elif -4 <= val <= -2:
        return 6
    elif -2 < val < 0:
        return 7
    elif 0 <= val < 2:
        return 8
    elif 2 <= val < 4:
        return 9
    elif 4 <= val < 6:
        return 10
    elif 6 <= val < 10:
        return 11
    elif 10 <= val < 15:
        return 12
    elif 15 <= val < 20:
        return 13
    elif val > 20:
        return 14
    else:
        print("cpi " + str(val) + "not assigned a box")

def get_bucket_exch(val):
    if val < -20:
        return 15
    elif -20 <= val < -15:
        return 16
    elif -15 <= val < -10:
        return 17
    elif -10 <= val < -8:
        return 18
    elif -8 <= val <-6:
        return 19
    elif -6 <= val < -4:
        return 20
    elif -4 <= val < -2:
        return 21
    elif -2 <= val < 0:
        return 22
    elif 0 <= val < 2:
        return 23
    elif 2 <= val < 4:
        return 24
    elif 4 <= val < 6:
        return 25
    elif 6 <= val < 10:
        return 26
    elif 10 <= val < 15:
        return 27
    elif 15 <= val < 20:
        return 28
    elif val > 20:
        return 29
    else:
        return
        print("cpi " + str(val) + "not assigned a box")

def get_bucket_gdp(val):
    if 0 < val < 0.1:
        return 30
    elif 0.1 <= val < 0.2:
        return 31
    elif 0.2 <= val < 0.3:
        return 32
    elif 0.4 <= val < 0.3:
        return 33
    elif 0.5 <= val < 0.4:
        return 34
    else:
        return
        print("gdp " + str(val) + " not assigned a bucket")

def get_boolean_states(v):
    nodes = []
    if v[0]:
        nodes.append(35)
    if v[1]:
        nodes.append(36)
    if v[2]:
        nodes.append(37)
    if v[3] == "crisis":
        nodes.append(38)
    if v[4]:
        nodes.append(39)
    if v[5]:
        nodes.append(40)
    if v[6]:
        nodes.append(41)
    return nodes

num_nodes = 42

#evaluators
crisis_ev =  lambda x: max(x[37], x[38],x[39]) + max(x[12],x[13],x[14], 0) + max(x[30], x[31]) + max(x[26], x[27], x[28], x[29]) + max(x[40], x[41]) 
crisis_ev2 = lambda x: max(x[40], x[41], x[37], x[38], x[39]) + max(x[0], x[1]) + max(x[15], x[16]) +  max(x[30], x[31])+ max(x[28], x[29]) 
crisis_ev3 = lambda x: max(x[40], x[41], x[37], x[38], x[39], 0) + max(x[0], x[1], x[2], x[3], x[4], x[5], 0) +  max(x[30], x[31], 0)+ max(x[21], x[22], x[23], 0) 
crisis_ev4 = lambda x: max(x[40], x[41], x[37], x[38], x[39], 0) + max(x[40], x[41], 0) +  max(x[30], x[31], 0)+max(x[0],x[1],x[1], x[2], x[3], x[5], 0) 
crisis_ev5 = lambda x: x[39] + max(x[40], x[41]) + max(x[10], x[11], x[12])
crisis_ev6 = lambda x: x[39] + x[40] + x[41]  +x[37] + max(x[6], x[7], x[8], x[9], x[10])
downturn_response_ev = lambda x: max(x[12],x[13],x[14]) + max(x[26], x[27], x[28], x[29]) + x[30] + x[31]
upturn_response_ev = lambda x: max(x[27], x[28], x[29]) + max(x[0],x[1],x[2], x[3], x[4]) +  max(x[32], x[33])
upturn_no_crisis_ev = lambda x: max(x[22], x[23], x[24], x[25]) + max(x[4],x[5],x[6]) - x[40] - x[41] - x[37] - x[38]
downturn_no_crisis_ev = lambda x: max(x[22], x[23], x[24]) + max(x[7],x[8],x[9])- x[40] - x[41] - x[37] - x[38] 
independence_banking_ev = lambda x: x[39] + max(x[35]+x[36], x[35], x[36], 0) + max(x[22], x[23], 0) + max(x[0], x[1], x[2], x[3], x[4], x[5]) + x[30] 
downturn_ev = lambda x: max(x[22], x[23], x[24], x[25]) + max(x[7],x[8],x[9],x[10],x[11]) +  max(x[30], x[31])+  max(x[40], x[41], x[37], x[38]) + x[40] + x[41] 
upturn_ev = lambda x: x[22]+ x[23]+ x[24]+ x[25]+ x[27]+ x[28]+ x[29] +x[4]+x[5]+x[6] +x[35]
updown_ev = lambda x: max(x[15], x[16], x[17], x[18], x[19], x[20], x[21]) + max(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7])
downup_ev = lambda x: max(x[22], x[23], x[24], x[25], x[26],x[27], x[28]) + max(x[8], x[9], x[10], x[11], x[12], x[13], x[14]) +x[35]
currency_banking_crisis_ev = lambda x: max(0, x[30]) + max(0, x[36]) + max(0, x[39]) + max(0, x[6],x[7],x[8],x[9],x[10],x[11]) 
evaluators = [crisis_ev, crisis_ev2, crisis_ev3, crisis_ev4, crisis_ev5, crisis_ev6, downturn_response_ev, upturn_response_ev, upturn_no_crisis_ev, downturn_no_crisis_ev,  
              independence_banking_ev, downturn_ev, upturn_ev,  updown_ev, downup_ev, currency_banking_crisis_ev]

crisis_fn =  lambda x: max(x[37], x[38],x[39]) + max(x[12],x[13],x[14], 0) + max(x[30], x[31]) + max(x[26], x[27], x[28], x[29]) + max(x[40], x[41]) > 3
crisis_fn2 = lambda x: max(x[40], x[41], x[37], x[38], x[39]) + max(x[0], x[1]) + max(x[15], x[16]) +  max(x[30], x[31])+ max(x[28], x[29]) > 3
crisis_fn3 = lambda x: max(x[40], x[41], x[37], x[38], x[39], 0) + max(x[0], x[1], x[2], x[3], x[4], x[5], 0) +  max(x[30], x[31], 0)+ max(x[21], x[22], x[23], 0) > 3
crisis_fn4 = lambda x: max(x[40], x[41], x[37], x[38], x[39], 0) + max(x[40], x[41], 0) +  max(x[30], x[31], 0)+max(x[0],x[1],x[1], x[2], x[3], x[5], 0)  > 3
crisis_fn5 = lambda x: x[39] + max(x[0], x[1], x[2], x[3], x[4],x[5],x[6])>=  2
crisis_fn6 = lambda x: x[39] + x[40] + x[41]+x[37] +x[35] +max(x[6], x[7], x[8], x[9], x[10])>2
downturn_response_fn = lambda x: max(x[12],x[13],x[14]) + max(x[26], x[27], x[28], x[29]) + x[30] + x[31]>2
upturn_response_fn = lambda x: max(x[27], x[28], x[29]) + max(x[0],x[1],x[2], x[3], x[4]) +  max(x[32], x[33])> 1
upturn_no_crisis_fn = lambda x: max(x[22], x[23], x[24], x[25]) + max(x[4],x[5],x[6]) - x[40] - x[41] - x[37] - x[38] == 2
downturn_no_crisis_fn = lambda x: max(x[22], x[23], x[24]) + max(x[7],x[8],x[9])- x[40] - x[41] - x[37] - x[38] == 2
independence_banking_fn = lambda x: x[39] + max(x[35]+x[36], 0) + max(x[22], x[23], 0) + max(x[0], x[1], x[2], x[3], x[4], x[5]) >= 3
downturn_fn = lambda x: max(x[22], x[23], x[24], x[25]) + max(x[7],x[8],x[9],x[10],x[11]) +  max(x[30], x[31])+  max(x[37], x[38]) + x[40] + x[41] > 2
upturn_fn = lambda x: x[22]+ x[23]+ x[24]+ x[25]+ x[27]+ x[28]+ x[29] +x[4]+x[5]+x[6]+x[35] >  2
updown_fn = lambda x: max(x[15], x[16], x[17], x[18], x[19], x[20], x[21]) + max(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7]) +x[35] >= 2
downup_fn = lambda x: max(x[22], x[23], x[24], x[25], x[26],x[27], x[28]) + max(x[8], x[9], x[10], x[11], x[12], x[13], x[14])+x[35] >= 2
currency_banking_crisis = lambda x: max(0, x[30]) + max(0, x[36]) + max(0, x[39]) + max(0, x[6],x[7],x[8],x[9],x[10],x[11]) >= 2

state_fxns = [crisis_fn, crisis_fn2, crisis_fn3, crisis_fn4, crisis_fn5, crisis_fn6, downturn_response_fn, upturn_response_fn, upturn_no_crisis_fn, downturn_no_crisis_fn,  
              independence_banking_fn,   downturn_fn, upturn_fn,  updown_fn, downup_fn, currency_banking_crisis]

#state_b = [3,3,3,3,2,2,2,1,2,2,3,2,2,2,2,2]
state_b = [3,3,3,3,2,2,2,2,2,2,3,4,2,2,2,2]
state_dict = {0:"overall_crisis", 1:"index_crisis", 2:"general_crisis", 3:"wide_crisis", 4:"crisis_with_debt", 5:"banking_crisis", 6:"downturn_response",
             7:"upturn_response", 8:"upturn_normal", 9:"downturn normal",10: "banking_independence",
             11:"downturn_normal", 12:"upturn_normal", 13:"increasing_cpi", 14:"increasing_exch", 15:"currency_banking_crisis"}


In [8]:
def state_selection(state_res):
    try:
        indices = state_res.index(True)
    except Exception as e:
       # print(state_res)
        return len(state_res)-1
    return indices

def conditional_prob(probs):
   # print(len(probs), probs)
    allres = [[] for i in range(len(probs))]
    for i in range(len(probs)):
        p = probs[i]
        if not p:
            continue
        total = p[-1]
        state_res = []
        if total:
            for ii in range(len(p)-1):
                state_res.append(p[ii] / total)
        else:
            state_res = [0 for x in range(len(p)-1)]
        allres[i] = state_res
    return allres

def state_prob(cprobs, allobs):
    res = []
    for i in range(len(cprobs)):
        x = cprobs[i]
     #   print(cprobs[i])
        if not x:
            res.append(0)
            continue
        res.append(round(x[-1] / allobs, 2))
    return res

def get_hmm_from_pairs(pairs, num_states):
    scratch = [[0 for i in range(num_states+1)] for i in range(num_states+1)]
    res = [[0 for i in range(num_states)] for i in range(num_states)]
    
    for p in pairs:
        scratch[p[0]][p[1]] += 1
        scratch[p[0]][-1] += 1
        
    for i in range(num_states):
        for j in range(num_states):
            if scratch[i][-1]:
                res[i][j] = scratch[i][j] / scratch[i][-1]
            
    return res

def encoding_row(row, indices):
   
    nodes = []
    p_currency_crises, p_inflation_crises = 36, 37
    p_domestic_default, p_sovereign_default = 40, 41
    p_systematic_crises, p_banking_crises = 38, 39 
    p_independence = 35
    positives = [0 for i in range(num_nodes+1)]
    unit_cpi= row['unit_cpi']
    b1 = get_bucket_cpi(unit_cpi)
    if b1:
        nodes.append(b1)
    unit_exch = row['unit_exch']
    b2 = get_bucket_exch(unit_exch)
    if b2:
        nodes.append(b2)
    gdp_weighted = row['gdp_weighted_default_x']
    b3 = get_bucket_gdp(gdp_weighted)
    if b3:
        nodes.append(b3)
        
    independence = row['independence']
    currency_crises = row['currency_crises']
    inflation_crises = row['inflation_crises']
    banking_crises = row['banking_crisis']
    domestic_default = row['domestic_debt_in_default']
    sovereign_default = row['sovereign_external_debt_default']
    systemic_crisis = row['systemic_crisis']
    v = [independence, currency_crises, inflation_crises, banking_crises, systemic_crisis, domestic_default, sovereign_default]
    positives[p_currency_crises] = currency_crises
    positives[p_inflation_crises] = inflation_crises
    positives[p_domestic_default] =  domestic_default
    positives[p_sovereign_default] = sovereign_default
    positives[p_systematic_crises] = systemic_crisis
    positives[p_banking_crises] = banking_crises == "crisis"
    positives[p_independence] = independence

    relevant = nodes  
        
    for i in range(len(v)):
        r = v[i]
        if r:
            relevant.append(indices[i])
    nodes.extend(get_boolean_states(v))
    
    return positives, list(set(nodes))
    

def forming_tab(d4, num_nodes,state_fxns, state_obs):
    m = [[0 for i in range(num_nodes+1)] for i in range(num_nodes+1)]
    num_edges = 0
    p_currency_crises, p_inflation_crises = 36, 37
    p_domestic_default, p_sovereign_default = 40, 41
    p_systematic_crises, p_banking_crises = 38, 39 
    p_independence = 35
    positives = [0 for i in range(num_nodes+1)]
    indices = [p_independence, p_currency_crises, p_inflation_crises, p_banking_crises, p_systematic_crises, p_domestic_default, p_sovereign_default]
    pairs = []
    crisis_trues = 0
    probs_under_states, next_probs_under_states = [[0 for i in range(num_nodes+1)] for i in range(len(state_fxns)+1)],  [[0 for i in range(num_nodes+1)] for i in range(len(state_fxns)+1)]
    all_res = []
    prev_state = -1
    history_in_obs, years = [], []
    for i in range(len(d4)-1):
        
        state_fn_res = []
        years.append(d4.iloc[i]['year'])
        positives, nodes= encoding_row(d4.iloc[i], indices)
        positives_next, nodes_next =  encoding_row(d4.iloc[i+1], indices)
        history_in_obs.append(nodes)
        gg = []
        for i in range(len(state_fxns)):
            f = state_fxns[i]
            state_fn_res.append(f(positives))
            ff = evaluators[i]
            gg.append(state_eval(i, positives,state_b))
       # if not any(gg):
           # print(nodes)
           # print("can't assign state")
    
        res = state_selection(gg)
       # print(gg)
        all_res.append(res)
        
        for i in range(len(positives)):
            within_def = all_state_obs[res]
            
          #  if i not in within_def:
            probs_under_states[res][i] += min(1, positives[i])
            #else:
                
            #    probs_under_states[res][i] = 0
        probs_under_states[res][-1] += 1 
       # print(res, probs_under_states)
        for i in range(len(positives_next)):
            within_def = all_state_obs[res]
            #if i in within_def:
            next_probs_under_states[res][i] += min(1, positives[i])
        next_probs_under_states[res][-1] += 1
        
        for i in nodes:
            for j in nodes:
                if i == j:
                    continue
                else:
                    m[i][j] += 1
                    m[j][i] += 1
        for j in nodes:
            m[j][j] -=1
        
        a = min(prev_state, res)
        b = max(prev_state, res)
        pairs.append((a,b))
        prev_state = res
   
    positives[-1] = len(d4)
    probs = conditional_prob(probs_under_states)
    next_probs = conditional_prob(next_probs_under_states)
    freq_state = state_prob(probs_under_states, len(d4))
    hmm = get_hmm_from_pairs(pairs, len(state_fxns))
    concurrent_occurences = m
    history_in_obs.append(nodes_next)
 #   print(all_res)
    return m, history_in_obs, years, next_probs, probs

m, history,years, nprob, cprob = forming_tab(d4, num_nodes, state_fxns, all_state_obs)
print(cprob)

NameError: name 'state_eval' is not defined

In [18]:
def dissect(obs_history, seq_len):
    res, i = [], 0
    while i < len(obs_history):
        res.append(obs_history[i:i+seq_len])
        i+=seq_len-1
    return res

def find_states_connected(obs, state_obs):
    available_states = []
    
    for i in range(len(state_obs)):
        if obs in state_obs[i]:
            available_states.append(i)
#    print(obs, state_obs, available_states)
    return available_states


processed = dissect(history, 5)
print(processed[0])
find_states_connected(5, all_state_obs)


NameError: name 'history' is not defined

In [10]:
import random
import math


def ve_single_hmm(state1, state2, observation_ind, state_obs):
    #ve_single要重新给hmm写
    
    a = hmm[state1][observation_ind]
    b = hmm[state2][observation_ind]
    
    if not a or not b:
        return -1
    
   # return min(a, b)
    return (a*b)

def ve_single(state1, state2, observation_ind, cprob):
    #ve_single要重新给hmm写
    
    if not cprob[state1] or not cprob[state2]:
    #    print("error in ve_single")
        return -1
    a = cprob[state1][observation_ind]
    b = cprob[state2][observation_ind]


    xs, ys = [], []
   # print('poi', a, b)
    for i in range(10):
        xs.append(random.uniform(0, a))
        ys.append(random.uniform(0, b))
        
    x, y = np.mean(xs), np.mean(ys)
    factor = 1000
   # print(len(cprob[state1]), state1, state2, observation_ind, x, y)
   # return min(x, y)
    if not x or not y:
        return 1
    return 1/(x*y)

def ve_obs(obs_in, num_states, cprobs, all_states, ishmm):
    f = [[0 for i in range(len(cprobs))] for i in range(len(cprobs))]
    source, sink, middle_node, first = [], [], [], True
    middle = False
    
    for ob in obs_in:
        #找出所有跟这个o有关的state
        sink = []
        ob_instance = list(ob)
        iii = 0
      #  print("sdfds", ob_instance)
        for inst in ob_instance:            
            connected_states = find_states_connected(inst, all_states)
            if  iii== int(round(len(ob_instance) / 2)):
                middle = True
            iii += 1
           # print("c", connected_states, f)
            for s in connected_states:
                f[s][s] += 1
                if first:
                    source.append(s)
                elif middle:
                    middle_node.append(s)
                else:
                    sink.append(s)
            connected_states2 = connected_states
            for i in connected_states:
                for j in connected_states2:
                    wij = ve_single(i, j, inst, cprobs)
                    
                    if wij > 0:

                        f[i][j] += math.sqrt(wij)
                        f[j][i] += math.sqrt(wij)
                            
        first, middle = False, False
    
    source = list(dict.fromkeys(source))
    sink = list(dict.fromkeys(sink))
    middle_node = list(set(middle_node))
    return f, source, middle_node, sink

In [11]:
def get_edge_weight(observation, state, cprobs):
    try:
        xs = []
   # print(state, observation)
        cc = cprobs[state]
    
        if not cc:
            return 1
        a = cprobs[state][observation]
        for i in range(10):
            xs.append(random.uniform(0, a))
        factor = 10
        return factor * np.mean(xs)
    except Exception as ee:
        print(observation)
        print(ee)
        return 1
    
def belong_to_same_state(obs1, obs2, cprobs):
    b = False
    for cp in cprobs:
        if obs1 in cp and obs2 in cp:
            b = True
        
    if b:
        return 1.625
    return 1.375

def left_triangle_prod(tup1, tup2, cprobs):
    prod = 1
   
    for e1 in tup1:
        for e2 in tup2:  
            weight1, node1 = e1[0], e1[1]
            weight2, node2 = e2[0], e2[1]
            y = belong_to_same_state(node1, node2, cprobs)
            weight1 = max(weight1, 1)
            weight2 = max(weight2, 1)
            prod *= y*weight1*weight2
           # print(prod, weight1, weight2, y)
    return prod

def right_triangle_prod(tup1, tup2):
    prod = 1
    for e1 in tup1:
        for e2 in tup2:  
            weight1, node1 = e1[0], e1[1]
            weight2, node2 = e2[0], e2[1]
            
            y = 1 # hmm[node1][node2]
            weight1 = max(weight1, 1)
            weight2 = max(weight2, 1)
            prod *= y*weight1*weight2
    return prod

def print_ordering(order):
    x = ""
    for s in order:
        x = x + state_dict[s] + ","
    return "{"+ x[:-1] + "}"

In [12]:
set1 = [(14,17, 41), (8,29, 41), (8,6,19,29, 41),(6,19, 38),(6,19, 38),(29, 8, 37, 35), (29, 8,36,37, 35)]
ex_seq = [12, 10, 2, 1, 3]

def encode_obs_set(ls):
    x = [0 for x in range(num_nodes+1)]
   # print(ls)
    for i in ls:
        x[i] = 1
    return x

def state_eval(state, xinst,state_b):
    ev = evaluators[state]
    b_diff = state_b[state] - ev(xinst)
    return b_diff == 0 or b_diff == 1

def stateseq_feasibility_eval(states, obs_set, state_b):
    for i in range(min(len(states), len(obs_set))):
        s = states[i]
        obs = obs_set[i]
        xinst = encode_obs_set(obs)
        if not state_eval(s, xinst, state_b):
            return False
    return True

def filter_seqs(all_seqs, obs_set, state_b):
    res = []
    for seq in all_seqs:
        if stateseq_feasibility_eval(seq, obs_set, state_b):
            res.append(seq)
    return res
#for i in range(len(evaluators)):
#    print(i, state_eval(i, encode_obs_set(set1[0]), state_b))   

In [13]:
#Given a list of events, we want the pattern of this country to explain this list of events
#x = overall_probability([0,1,2],num_nodes, )
#要构一个hidden states的markov chain, subset_probability 用来做大状态到小状态的边的权重
#a = get_edge_weight("inflation_crisis", crisis_states)
#set1 = [(23, 8, 35), (23, 6, 35), (29, 8,36, 35),(29, 8, 37, 35), (29, 8,36,37, 35)]

def mnker(obs_set, tested_seq, m, cprobs, nprobs, all_states):
    first = True
    products = 1
    prev_diag = []
    
    for i in range(len(obs_set)-1):
        ob = obs_set[i]
        edges1, edges2 = [], []
        left_product = 1
        
        if i < len(tested_seq):
            state = tested_seq[i]
        else:
            break
            
        for inst in ob:
            if isinstance(state, list):
                for iii in inst:
                    edges1.append((get_edge_weight(iii, state, cprobs), iii))
                    
            else:
                edges1.append((get_edge_weight(inst, state, cprobs), inst))
        ob2 = obs_set[i+1]
        for inst2 in ob2:
            edges2.append((get_edge_weight(inst2, state, nprobs), inst2))
        left_product = left_triangle_prod(edges1, edges2, cprobs)
        right_product = 1
        if not first:
          #  print(tested_seq)
            right_product = right_triangle_prod(prev_diag, edges2)
            
        products += left_product + right_product
        prev_diag = edges2
        first = False
    return products   
#hmm = get_hmm2(all_states, prob_under_states)
mnker(set1, ex_seq, m, cprob , nprob, all_state_obs)

NameError: name 'm' is not defined

In [14]:
import itertools
def get_all_seqs(all_states, length):
    xs = [i for i in range(len(all_states)-2)]
    combination_objects = itertools.combinations(xs, length)
    all_perms = []
    for x in combination_objects:
        permutations_object = itertools.permutations(x)
        permutations_list = list(permutations_object)
        all_perms.extend(permutations_list)
    return list(all_perms)

def identifying_best(oset, all_seq, m, cprobs,nprobs, all_states):
    max_prod, best = 0, []
    for seq in all_seq:
        prod = mnker(oset, seq, m, cprobs, nprobs, all_states)
        if prod > max_prod:
            best = seq
            max_prod = prod
         #   state_ordering = print_ordering(best)
   # print(best, max_prod)
    return best, max_prod

def overall(cases):
    e = get_subdf(cases[0])
    for c in case_num[1:]:
        d = get_subdf(c)
        e = pd.concat([e, d])
    return e

stepper1 =  [2, 2, 2, 0, 0, 2, 3, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2]
hmm1 =  [658.2302995208356, 198.99139140982965, 1, 3.5830479975109237, 1, 18.425969233137117, 817.8100132973431, 192.40050844974851, 0.952544015446258, 1, 1, 216.8341253201385, 1, 1, 0.4271133641069199, 1, 170.35695513444617, 1]
cprobs1 = [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.5, 0.5, 0.5, 1.0, 0.0, 0.0], [0.42857142857142855, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8571428571428571, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.42857142857142855, 0.42857142857142855, 0.0, 0.14285714285714285, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0], [0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5], [], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.05555555555555555, 0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1111111111111111, 0.5, 0.1111111111111111, 0.0, 0.05555555555555555, 0.05555555555555555, 0.0, 0.0, 0.6666666666666666, 0.0, 0.0, 0.0, 0.0, 0.8333333333333334, 0.0, 0.3333333333333333, 0.2222222222222222, 0.3888888888888889, 0.0, 0.16666666666666666], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.43478260869565216, 0.2608695652173913, 0.30434782608695654, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08695652173913043, 0.6521739130434783, 0.21739130434782608, 0.043478260869565216, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.42857142857142855, 0.5, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.03571428571428571, 0.8928571428571429, 0.07142857142857142, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [], [0.23076923076923078, 0.0, 0.15384615384615385, 0.038461538461538464, 0.0, 0.07692307692307693, 0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.07692307692307693, 0.11538461538461539, 0.07692307692307693, 0.15384615384615385, 0.0, 0.0, 0.038461538461538464, 0.0, 0.0, 0.0, 0.07692307692307693, 0.07692307692307693, 0.5769230769230769, 0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.15384615384615385, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.15384615384615385, 0.038461538461538464, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.6, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [], [0.0, 0.0, 0.10526315789473684, 0.15789473684210525, 0.05263157894736842, 0.21052631578947367, 0.21052631578947367, 0.2631578947368421, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21052631578947367, 0.05263157894736842, 0.10526315789473684, 0.0, 0.05263157894736842, 0.21052631578947367, 0.3684210526315789, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.05, 0.0, 0.0, 0.05, 0.0, 0.35, 0.25, 0.0, 0.1, 0.05, 0.0, 0.0, 0.15, 0.2, 0.0, 0.0, 0.0, 0.0, 0.05, 0.3, 0.0, 0.0, 0.05, 0.0, 0.0, 0.0, 0.0, 0.4, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], []]
obs1 = [23, 29, 30, 35, 7, 8, 29, 30, 35, 36, 37, 39, 14, 30, 35]
obs2 = [15, 28, 30, 35, 7, 9, 15, 29, 30, 35, 36, 37, 39, 12, 29, 30, 35]

def append_obs(cprobs, state, obs, r):
    if cprobs:
        for i in range(len(cprobs)):
            
            if r < cprobs[i]:
                obs.append(i)
    return obs

def get_state(obs, state_fxns):
    x = [0 for i in range(num_nodes)]
    for o in list(set(obs)):
        x[o]  = 1
    state_fn_res = []
    
    for i in range(len(state_fxns)):
        f = state_fxns[i]
        state_fn_res.append(f(x))
    
    res = state_selection(state_fn_res)
  #  print("get_state", res, state_fn_res)
    return res

def print_history(ordered, years):
    res = ""
    ordered = ordered[1:-1]
    states = ordered.split(",")
    for i in range(len(years)):
        res += str(years[i])+ ":" + str(states[i]) + "; "
    return res[:-1]
    
def extrapolate(cprobs, hmm, stepper, last):
    most_likely_observations = []
    r = random.uniform(0,1)
    cp = cprobs[last]
    most_likely_observations = append_obs(cp, last, most_likely_observations, r)
    a = get_state(most_likely_observations, state_fxns)
    x = np.argmax(hmm)
    y = np.argmax(stepper1)
    
    most_likely_observations = append_obs(cprobs[x], last, most_likely_observations, r)
    most_likely_observations = append_obs(cprobs[y], last, most_likely_observations, r)
    most_likely_state = get_state(most_likely_observations, state_fxns)
    print(print_ordering([x]), print_ordering([y]) ,print_ordering([a]), print_ordering([most_likely_state]), most_likely_observations)

extrapolate(cprobs1, hmm1, stepper1, 5)
get_state(obs2, state_fxns)

{downturn_response} {downturn_response} {currency_banking_crisis} {increasing_exch} [30, 35, 14, 30, 35, 14, 30, 35]


0

In [19]:
state_seq_length = 6
import networkx as nx
from sklearn.preprocessing import normalize

def best_for_case(data, num_nodes, length, observation_set, all_states, state_fxns, states_b):
    
    m, history, years, nprobs, cprobs = forming_tab(data, num_nodes, state_fxns, all_states)
    all_seq = get_all_seqs(all_states, length)
   # print(cprobs)
    filtered_seq = filter_seqs(all_seq, observation_set, states_b)
    frames = dissect(history, length)
    cadj, src, middle, sink = ve_obs(observation_set, len(all_states), cprobs, all_states, False)
    nadj, src_1, middle_1, sink_1 = ve_obs(observation_set, len(all_states), nprobs, all_states, False)  #1step
    best_order, prod = identifying_best(observation_set, filtered_seq, m, cprobs, nprobs,all_states)
  #  extrapolate(cprobs, adj[best_order[-1]], adj_1[best_order[-1]], best_order[-1])
    best_order = print_ordering(best_order)
    
    return best_order, prod, cprobs, nprobs,cadj, nadj

def explaining_history(case, length, state_fxns, all_states, states_b):
    
    data = get_subdf(case)
    print("started for：",case, "there are ", len(data), "rows and ", int(len(data) / length), "frames. ")
    m, history, years, nprobs, cprobs = forming_tab(data, num_nodes, state_fxns, all_states)
   # heavy = trees(cprobs, nprobs)
    
    frames = dissect(history, length)
    years_frame = dissect(years, length)
    all_seq = get_all_seqs(all_states, length)
    history = []
    prods = []
    
    for i in range(len(frames) - 12,len(frames)):
        f = frames[i]
        y = years_frame[i]
        filtered_seq = filter_seqs(all_seq, f, states_b)
        best_order, prod = identifying_best(f, filtered_seq, m, cprobs, nprobs, all_states)
        best_order = print_ordering(best_order)
        history.append(best_order)
        prods.append(prod)
        printer = print_history(best_order, y)
        print(i, printer, f)
        
    print(history)
    return history, prods
    
def country_based(cases, all_states, state_fxns, states_b):
    print("started")
    for c in cases:
        d4 = get_subdf(c)
      #  best_order, prod, adj, cprobs, adj_1 = best_for_case(d4, num_nodes, state_seq_length, set1, all_states, state_fxns)
        best_order, prod, cprobs, nprobs, cadj, nadj = best_for_case(d4, num_nodes, state_seq_length, set1, all_states, state_fxns, states_b)
        print(str(c) + ":"+ str(best_order))
    d_all = overall(case_num)
    best_order, prod, cprob, nprob, cadj, nadj = best_for_case(d_all, num_nodes, state_seq_length, set1, all_states, state_fxns, states_b)
    print("all: "+ str(best_order))
    
#country_based(case_num, all_state_obs, state_fxns, state_b) 
explaining_history("EGY", state_seq_length, state_fxns, all_state_obs, state_b) 
#adj, src, middle, sink = ve_obs(set1, 42, cprob, all_states)
#print(cprobs)

# Extrapolation of values of observation, and from 1 step models. And then get the next state based on values of observation, and 1 step hmm model

started for： EGY there are  154 rows and  25 frames. 
19 1956:downturn_response; 1957:increasing_cpi; 1958:banking_crisis; 1959:crisis_with_debt; 1960:upturn_normal; 1961:general_crisis; [[35, 39, 14, 23], [25, 35, 39, 7], [35, 4, 39, 23], [35, 4, 39, 23], [8, 35, 39, 23], [9, 35, 39, 23]]
20 1961:downturn normal; 1962:upturn_normal; 1963:increasing_cpi; 1964:general_crisis; 1965:crisis_with_debt; 1966:banking_independence; [[9, 35, 39, 23], [35, 36, 29, 39], [35, 4, 39, 23], [35, 12, 39, 23], [9, 35, 39, 23], [35, 39, 6, 23]]
21 1966:general_crisis; 1967:crisis_with_debt; 1968:downturn normal; 1969:overall_crisis; 1970:increasing_cpi; 1971:banking_independence; [[35, 39, 6, 23], [39, 35, 5, 23], [35, 12, 39, 23], [35, 39, 14, 23], [35, 23, 39, 7], [35, 23, 39, 7]]
22 1971:banking_crisis; 1972:general_crisis; 1973:crisis_with_debt; 1974:upturn_normal; 1975:increasing_cpi; 1976:upturn_normal; [[35, 23, 39, 7], [35, 23, 39, 7], [35, 11, 39, 23], [10, 35, 39, 15], [35, 23, 39, 7], [8, 35,

(['{downturn_response,increasing_cpi,banking_crisis,crisis_with_debt,upturn_normal,general_crisis}',
  '{downturn normal,upturn_normal,increasing_cpi,general_crisis,crisis_with_debt,banking_independence}',
  '{general_crisis,crisis_with_debt,downturn normal,overall_crisis,increasing_cpi,banking_independence}',
  '{banking_crisis,general_crisis,crisis_with_debt,upturn_normal,increasing_cpi,upturn_normal}',
  '{upturn_normal,banking_crisis,general_crisis,banking_independence,crisis_with_debt,upturn_normal}',
  '{increasing_cpi,crisis_with_debt,banking_crisis,wide_crisis,upturn_normal,general_crisis}',
  '{banking_independence,downturn_normal,downturn normal,upturn_response,crisis_with_debt,banking_crisis}',
  '{crisis_with_debt,general_crisis,downturn_response,banking_crisis,upturn_normal,downturn normal}',
  '{upturn_normal,general_crisis,banking_independence,downturn normal,crisis_with_debt,index_crisis}',
  '{downturn normal,crisis_with_debt,wide_crisis,general_crisis,increasing_cpi,u

In [16]:
import networkx as nx
import matplotlib.pyplot as plt
num_nodes = 42
d_all = overall(case_num)

best_order, prod, cprob, nprob, cadj, nadj = best_for_case(d_all, num_nodes, state_seq_length, set1, all_state_obs, state_fxns, state_b)

print(len(cprob))

17


In [17]:
print(cprob)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.5571428571428572, 0.6571428571428571, 0.5428571428571428, 0.6142857142857143, 0.3142857142857143, 0.9857142857142858], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.06451612903225806, 0.0, 0.0, 0.0, 0.21505376344086022, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.22, 0.28, 0.8,

In [ ]:
def trees(cadj, nadj):
    A = np.triu(np.matrix(cadj, dtype = float))
    M = np.matrix(nadj, dtype = float)
    MA = np.multiply(M, A)
    AM = np.matmul(A, M)
    AA = np.matmul(A, A)
    AMA = np.matmul(AM, A)
    AAM = np.matmul(AA, M)
    AAMA = np.triu(np.multiply(A,AMA))
    T = nx.maximum_spanning_tree(nx.from_numpy_matrix(np.asmatrix(MA)))
    T2 = nx.minimum_spanning_tree(nx.from_numpy_matrix(np.asmatrix(AMA)))
    T3 = nx.maximum_spanning_tree(nx.from_numpy_matrix(np.asmatrix(AM)))
    nx.draw_networkx(T3)
    sorted(T3.edges(data=True))
    res = []
    for e in T.edges:
        res.append(e[0])
        res.append(e[1])
        
    return list(set(res))

In [ ]:
from networkx.algorithms.cycles import cycle_basis
cba = nx.cycle_basis(nx.from_numpy_array(A))

cbam = nx.cycle_basis(nx.from_numpy_array(MA))

outr = []
for cb in cba:
    if len(cb) >= 3:
        outr.append(cb)
print(outr)

In [ ]:
dt = [("capacity", int), ("weight", float)]

from networkx.algorithms.flow import boykov_kolmogorov
G = nx.from_numpy_array(AMA - MA)
nx.set_node_attributes(G, M, "capacity")

def flow_between_pairs(G, num_states):
    res = []
    for i in range(num_states):
        best_flow, best_start_end,a = -10, [], 0
        for j in range(num_states):
            if i!= j:
                R = boykov_kolmogorov(G, i, j)
                mincostFlow = nx.maximum_flow_value(G, i, j)
               # print(i, j, mincostFlow)
                if mincostFlow >= best_flow:
                    best_flow = mincostFlow
                    best_start_end.append(j)
                    a = R
        print(i, best_flow, best_start_end)
        res.append((best_flow, best_start_end))
    return res

AMAdt = np.array(AM, dtype=dt)
G = nx.from_numpy_matrix(AMAdt)
flow_between_pairs(G, len(state_fxns))

In [ ]:
from sklearn.preprocessing import normalize
amadt = normalize(AMA, axis=0, norm='l2')
AMAdt = np.array(amadt, dtype=dt)
G2 = nx.from_numpy_matrix(AMAdt)
flow_between_pairs(G2, len(state_fxns))